## Sequence modeling of time series
* Recurrent neural nets are popular when the data has a temporal component
* A popular sequence model is known as the "Long Short Term Memory" (LSTM) network
    * LSTM models are able to retain information across long time scales in the input sequence
* Here we will apply the LSTM to our EEG data

## Load the data
* The second and third dimensions are swapped to accommodate the upcoming model

In [ ]:
filename = '../data/EEG.mat' # the data file
import scipy.io as sio # the library that we use to load Matlab data into Python
data = sio.loadmat(filename)
x_train = np.swapaxes(data['X_train'],1,2)
x_test = np.swapaxes(data['X_test'],1,2)
y_train = data['Y_train']
y_test = data['Y_test']


### Creating the model
* We will work with the Keras LSTM class 
* A key parameter is the number of hidden units in our LSTM layer - here we start with 100 units
* We add a "dropout" parameter to try to mitigate overfitting

In [ ]:
import keras
input_shape=(128,64) # this tells Keras the shape of the data that will be passed to it for training/testing
n_classes=2
n_lstm_units=20
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.LSTM(n_lstm_units,dropout=0.2),
        keras.layers.Dense(n_classes, activation="softmax"),
    ]
)
model.summary()

### Compile the model to prepare for training
* We will use the same training parameters as previously

In [ ]:
batch_size = 128  # how many training examples in each "batch" used to compute gradient of loss function
epochs = 30 # how many passes through the data we want the fitting to take
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",keras.metrics.AUC()])

### Fitting the model
* Note that the LSTM is a complex architecture and is thus heavily parameterized
* We will pay close attention to possible overfitting

In [ ]:
model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs)

### Evaluating the sequence model
* The training performance was quite high, but as always, we need to test on unseen data

In [ ]:
metrics=model.evaluate(x_test,y_test)
print("Model accuracy = " + str(metrics[1]))
print("Model AUROC = " + str(metrics[2]))

### Interpreting the performance
* Note that while the performance is very good, it did not exceed that obtained with a convolutional model
* It may be argued that the information in EEG is in the motifs, and not embedded in a true sequence (order-dependence)
* In practice, the number of units and dropout parameter are extensively tuned on a "validation" set